In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Docker

## First containers

Let's check that docker is installed

In [ ]:
!docker version

Let's run our first container:

In [ ]:
!docker run ubuntu

We can run a `-t`erminal within a container in `-i`nteractive mode with:

In [ ]:
!docker run -ti ubuntu

On Jupyter we can't interact with the container shell. **Let's continue on our machine**

`docker run -ti ubuntu`

Let's now install a package with

`apt-get update && apt-get install figlet`

Let's run 

`figlet hello`

We can exit our container by `Ctrl+D` or 

`exit`

Let's run it again

`docker run -ti ubuntu`

And let's try to run 

`figlet hello`

The command is not installed because this is a fresh container.

## Images and Containers

![container layers](img/container-layers.jpg)

An **image** is a read-only filesystem.

A **container** is an encapsulated set of processes, running in a read-write copy of that filesystem.

`docker run` starts a container from a given image

There are 3 namespaces for images:
* Official (e.g. `ubuntu`)
* User (e.g. `jpetazzo/clock`)
* Self-hosted (e.g. `registry.example.com:5000/my-private/image`)

To see the images present on our host we can run

In [ ]:
!docker images

## Daemonized containers and logs

Before we ran a container in interactive mode.

Let's now launch one in the background with the flag `-d`aemonized

In [ ]:
!docker run -d jpetazzo/clock

In the output we can see the created container ID

We can list all our running containers with:

In [ ]:
!docker ps

We can stop containers with:

In [ ]:
!docker stop 348

We can see logs with:

In [ ]:
!docker logs 766

We can see the `--tail` of the logs and even `--follow` the output with:

In [ ]:
!docker logs --tail 1 --follow 766

`docker stop` will exit the containers 'gracefully'.

To force stop, we can:

In [ ]:
!docker kill 766

We can list `-a`ll containers with:

In [ ]:
!docker ps -a

And we can restore it with

In [ ]:
!docker start 766

## Dockerfile

We can design a docker image by writing a `Dockerfile`.

Let's replicate what we did before and build a `figlet` image.

In [ ]:
%cd figlet_image

In [ ]:
!git checkout -- Dockerfile
!cat Dockerfile

`FROM` indicate the base image

`RUN` executes commands during the Docker build. 

Let's now `build` our image.

The syntax is `docker build 'building_context' -t 'tag_name'`.

In [ ]:
!docker build . -t figlet

In this case the building context is the directory `figlet_image` and we tagged the built image with `figlet`. 

Each `RUN` command is run in a container. The output of that container is saved in a new image.

Let's run the image on our shell with

`docker run -ti figlet`

and run 

`figlet hello`

If you build the image again it will be instantaneous:

In [ ]:
!docker build . -t figlet

The intermediate images obtained by running a certain **sequence** are saved in the Docker cache.

If you change a line so slightly in the Dockerfile, the cache will be broken.

## CMD and ENTRYPOINT

We want to print a welcome message when the image is run.

`CMD` defines an **overwritable** default command.

Let's add to our `Dockerfile` the line
```
CMD figlet -f script hello
```

In [ ]:
!echo '\nCMD figlet -f script hello' >> Dockerfile
!cat Dockerfile

Let's build again and run

In [ ]:
!docker build . -t figlet
!docker run figlet

We can override the `CMD` command by specifying a command after we run the image:

In [ ]:
!docker run -ti figlet bash

We now want to be able to specify our own string when running the image.

`ENTRYPOINT` specifies a **non-overwritable** base command.

With `CMD` we can define the default parameters.

Let's add the following lines to the `Dockerfile`:

```
ENTRYPOINT ["figlet", "-f", "script"] 

CMD ["hello world"]
```

In [ ]:
!echo '\nENTRYPOINT ["figlet", "-f", "script"] \nCMD ["hello world"]' >> Dockerfile
!cat Dockerfile

Only the last `CMD` and `ENTRYPOINT` entries are considered.

In [ ]:
!docker build . -t figlet
!docker run figlet Maersk

If now we want to `--override` the entrypoint, we can do:

In [ ]:
!docker run  -ti --entrypoint bash figlet

## Multi-stage builds

We now want to compile a C application and ship the compiled version in a container.

In [ ]:
%cd ../c_image

In [ ]:
!cat hello.c

In [ ]:
!git checkout -- Dockerfile
!cat Dockerfile

We use `ubuntu` as base image and we install `build-essential` as it includes a C compiler.

When we do `RUN apt-get install` we specify `-y` to give our consent for dependencies installation since interaction is not permitted.

We copy the source to the container, we compile it and we run the compiled binary.

In [ ]:
!docker build . -t hello

In [ ]:
!docker run hello

In the resulting image we have:
* Binary
* Source code
* Compiler

We actually need to ship only the first one. 

We can't just append `RUN make clean` or `RUN apt-get remove` because they would only be added as further layers, **not reducing image size**.

One solution to this is **collapsing layers**. We could write:
```
FROM ubuntu
RUN apt-get update \
 && apt-get install xxx \
 && ... \
 && apt-get remove xxx \
 && ...
 ```
This would result in one unique layer, not increasing image size.

Cons:
* Not very readable
* Expensive layer
* Some files might remain if cleanup is not thorough

A better solution is using multi-stage build. 
We can add another `FROM` to create a new stage.
We can `COPY` files from one stage to the other.

In [ ]:
!echo '\nFROM ubuntu\
\nCOPY --from=0 /hello /hello\
\nCMD /hello' >> Dockerfile
!cat Dockerfile

We can also name the stages with e.g. `FROM ubuntu as compiler`.

We can refer to it later with `COPY --from=compiler`.

Let's build the image with a new tag and compare image sizes.

In [ ]:
!docker build . -t hello:multistage
!docker run hello:multistage

We can compare image sizes with:

In [ ]:
!docker images hello

In [ ]:
!docker images ubuntu

## Efficient Dockerfiles: dependencies and unit tests

```
FROM python
WORKDIR /src
COPY . .
RUN pip install -qr requirements.txt
EXPOSE 5000
CMD ["python", "app.py"]
```

Using this `Dockerfile`, dependencies are installed every time.

```
FROM python
COPY requirements.txt /tmp/requirements.txt
RUN pip install -qr /tmp/requirements.txt
WORKDIR /src
COPY . .
EXPOSE 5000
CMD ["python", "app.py"]
```

This stores an image in cache with dependencies installed and only runs the python script.

When a change in `requirements.txt` is detected, the cache is broken and new dependencies are installed.

We can apply the same concept with unit tests:

```
FROM <baseimage>
RUN <install dependencies>
COPY <code>
RUN <build code>
RUN <install test dependencies>
COPY <test data sets and fixtures>
RUN <unit tests>
FROM <baseimage>
RUN <install dependencies>
COPY <code>
RUN <build code>
CMD, EXPOSE ...
```

If `RUN <unit tests>` fails, it will not build the image. If it succeeds, it will ship a clean image without test data and libraries.

## Networking

### Exposing containers

We want to run a web server in a container and access it from our host.

In [ ]:
!docker run -d -P nginx

`-P` stands for Publish and will expose the web server.

To see where it is running we can:

In [ ]:
!docker ps

The web server runs on port 80 inside the container and is mapped on port 32771 on our host.

We can access the web server by accessing http://localhost:32771.

We can specify the mapping of ports ourselves.

In [ ]:
!docker run -d -p 30123:80 nginx

In [ ]:
!curl localhost:30123

### Virtual networks

We now want to run an app which requires two containers connected to the same network.

<img src="img/networking.png" alt="Networking" style="width: 800px;"/>

In [ ]:
!docker network create dev

Let's run a simple Flask Python web server (https://github.com/jpetazzo/trainingwheels/blob/master/www/Dockerfile) which counts the number of received requests.

The requests are stored in a redis database.

In [ ]:
%cd ../trainingwheels/www

In [ ]:
!cat Dockerfile

In [ ]:
!cat counter.py

In [ ]:
!docker build . -t trainingwheels
!docker run --net dev -d -P trainingwheels

Let's see status of the `-l`ast container started:

In [ ]:
!docker ps -l

We can see the webapp at http://localhost:32772.

The error message says that the `redis` service is unknown.

Let's start a `redis` database connected to the same network.

In [ ]:
!docker run --net dev --net-alias redis -d redis

### Ambassador design pattern

<img src="img/ambassador.png" alt="Ambassador" style="width: 800px;"/>

DB ambassador:
- The database container is moved (or a failover happens). Its new location will be tracked by the ambassador container and the web application container will still be able to connect, without reconfiguration.
- The web application code does not have credentials. They are passed to the ambassador to perform authentication before forwarding traffic to the DB container

Web host ambassador:
- (When running multiple web containers) Run a load balancer and dispatch requests across all backends

## Volumes

### Local development in Docker

We want to deploy an application in a container, change the source files and see the changes.

In [ ]:
%cd ../../namer

In [ ]:
!cat Dockerfile

In [ ]:
!cat company_name_generator.rb

In [ ]:
!docker build . -t namer

In [ ]:
!docker run -dP namer
!docker ps -l

We now want to edit the code and see changes in real time.

Approaches:
* Develop locally and rebuild the image
* Install an editor in the container and develop inside
* Use volumes

In [ ]:
!docker run -dP -v $(pwd):/src namer
!docker ps -l

We have **bind mounted** our host's working directory to the container's `/src` folder. 

The files are not copied or synchronized.

If we change the local files the container will serve our updated local files.

To debug, we can `exec`ute a command in a running container with:

In [ ]:
!docker exec -ti 903 bash

### Shared volumes

Volumes can be created without being anchored to a specific path. They can be used by multiple containers.

If a container is stopped, its volumes still exist.

In [ ]:
!docker volume ls

In [ ]:
!docker volume rm 694984de315344a7c83a296de52307d29f141cd844abf09569aa59a256c68a16

Let's now create two empty volumes, to be mapped

In [ ]:
!docker volume create webapps

In [ ]:
!docker volume create logs

Let's now start a Tomcat web server using those two volumes.

In [ ]:
!docker run -d -p 30123:8080 \
         -v logs:/usr/local/tomcat/logs \
         -v webapps:/usr/local/tomcat/webapps \
         tomcat

In [ ]:
!curl localhost:30123

We can now start another container using the same volume.

The volume in the second container will be populated with the files written by the first container.

Let's run the below command on our machine:

`docker run -v webapps:/webapps -w /webapps -ti alpine vi ROOT/index.jsp`

Similarly we can see the logs by mounting the logs volume in another container.

`docker run -v logs:/logs -w /logs -ti alpine sh -c "tail -f /logs/*"`

Another easy example is with Database migration:

`docker run -d --name redis28 redis:2.8`

`docker run -ti --link redis28:redis busybox telnet redis 6379`

... Create data ...

`docker stop redis28`

`docker run -d --name redis30 --volumes-from redis28 redis:3.0`

`docker run -ti --link redis30:redis busybox telnet redis 6379`

... Data has been migrated!

### Sharing the Docker control socket

With the `-v` flag we can even share a single file. The most interesting example concerns the Docker control socket: the container sharing it will be connected to the Docker engine and will be able to issue `docker` commands.

`docker run -it -v /var/run/docker.sock:/var/run/docker.sock docker sh`

This container can now create containers. This is useful in the case where we want our CI system to run in a container AND it be able to start containers.

Read more here: https://jpetazzo.github.io/2015/09/03/do-not-use-docker-in-docker-for-ci/?source=post_page---------------------------

### Configuration volumes

Configuration is stored in a volume (can be shared across multiple containers).

Configuration is generated/updated by a configuration container.

The application container detects when configuration is changed and reloads when necessary.

See example at https://maersk-2019-07.container.training/three.yml.html#411

## Docker Compose

Docker Compose allows us to build, run and connect containers.

We define a `.yml` file describing the structure and we just run `docker-compose up`.

In [ ]:
%cd ../trainingwheels

In [ ]:
!cat docker-compose.yml

The structure of the file is always:
* version: "2"/"3"
* services:
    * service1:
        * build: path/to/Dockerfile
        * (optional) ports:
        * (optional) network:
        * (optional) command:
        * (optional) volumes:
    * service2:
        * image: name-of-image
        
By default, one private network gets created for each Compose file.

You can set environment variables, useful for debugging.

In [ ]:
!docker-compose --version

In [ ]:
!docker-compose up

In [ ]:
!docker-compose ps

In [ ]:
!docker-compose up --build

In [ ]:
!docker-compose down

You can set environment variables e.g. `DEBUG: 1`

# Kubernetes

## Why Orchestration?

Use case 1, highly available Web Server:

Auto scale up and down to accomodate pageviews.

![pageviews](img/pageviews.png)

Use case 2, scheduling in Data Center:

Rebalance VMs and shut down those which are not needed.

![binpacking](img/binpacking.gif)

A good orchestrator also needs to manage:
* Network between containers
* Load balancing
* Failure recovery
* Canary deployments
* Rolling updates

What can Kubernetes do?
* Start 5 containers using image `backend`
* Place an internal load balancer in front of these containers
* Start 10 containers using image `frontend`
* Place an internal load balancer in front of them
* Traffic spikes! Grow our cluster by deploying more containers
* New release! Deploy new `fronted` image on one container at a time, keeping the service available

## How does Kubernetes work?

![k8s-architecture](img/k8s-architecture.png)

On nodes:
* Container engine
* Kubelet agent to communicate with master node
* Kubeproxy to provide basic networking

On master:
* API server
* Core services: scheduler and controller manager
* `etcd`: highly available key-value database

Master coordinates creation of resources, e.g.:
* Nodes: physical/virtual machine in our cluster
* Pods: group of containers running on a node
* Services: network endpoint to connect to one or multiple containers

![resources](img/pod.png)

IP addresses get assigned to pods

## Kubectl, essential commands

`kubectl` is our way to interact with the API server on the master node.
It allows use to create and manage resources in a simple way.

In [ ]:
!kubectl version

The workshop at this point explains concepts such as deployments, replicaset and pods.
I diverge and focus on the workflow I as a developer use with K8S.

As a first suggestions, we should install something that shows us our current context.
I use a zsh plugin called `kubecontext`, which I enable and disable through my `.zshrc` file.

In [ ]:
!code ~/.zshrc

A **context** is a triplet defined by:
1. Cluster
2. Namespace
3. User

When we have configured them, we can see available contexts with:

In [ ]:
!kubectx

We can set our current context with:

In [ ]:
!kubectx digital-platform-batch-prod-west

A cluster is a group of nodes.

Namespaces are groups of resources. We can list namespaces **available in the current context** with:

In [ ]:
!kubectl get namespaces 

Or simply:


In [ ]:
!kubens

You can set your current namespace with:

In [ ]:
!kubens wondercast

You can then list pods within your current context with:

In [ ]:
!kubectl get pods

You can get the running pods by applying a selector:

In [ ]:
!kubectl get pods --field-selector=status.phase=Running

You can search for a string in the pod name with:

In [ ]:
!kubectl get pods | grep r2l

You can inspect logs associated with a certain pod with:

In [ ]:
!kubectl logs r2l-forecast-total--trigger-1564489800-847xv

To better read JSON logs we can use `jq`:

In [ ]:
!kubectl logs r2l-forecast-total--trigger-1564489800-847xv | jq